In [1]:
#### installing necessary libraries
# ! python3 -m spacy download en_core_web_lg
# ! pip install paddlepaddle == 2.0
# ! pip install paddleocr 
# ! pip install borb==2.0.27
# ! pip install pytesseract
# ! pip install ftfy
# ! pip install tesseract
# ! sudo apt install tesseract-ocr
# ! pip install keras-ocr
# ! python -m pip install spacy
# ! pip install nltk
# ! pip install pyforest
# ! sudo apt-get install tesseract-ocr
# ! pip install pytesseract==0.3.9
# ! python -m spacy train config.cfg --output ./models --paths.train ./train.spacy --paths.dev ./dev.spacy
# ! pip install datefinder
# ! pip install pillow
# ! pip install lexnlp
# ! pip install pdf2image
# ! pip install pytesseract

In [2]:
# print("Restarting of kernel...") 
# get_ipython().kernel.do_shutdown(True)

In [8]:
#### importing libraries
# !apt -qqy install xorg python3-pip
!pip install paddlepaddle paddleocr
!pip install -U numpy
from pyforest import *
import cv2
import numpy as np
import pytesseract
from PIL import Image
import imutils
from imutils.perspective import four_point_transform
import argparse
import re
from PIL import Image
from matplotlib import pyplot as plt
import os
from paddleocr import PaddleOCR
import random
%matplotlib inline
import sys
import nltk
import warnings
warnings.filterwarnings('ignore')
from borb.pdf.document.document import Document
import typing
from borb.pdf.pdf import PDF
from borb.toolkit.text.simple_text_extraction import SimpleTextExtraction
from borb.toolkit.ocr.ocr_as_optional_content_group import OCRAsOptionalContentGroup
from pathlib import Path
import ftfy
import unicodedata
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import pandas as pd
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#### path to files and dataset

path = '/content/drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Final Project/installation files/'
sys.path.append(path)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/content/drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Final Project/installation files/requirements/poppler-0.68.0/bin


In [5]:
# #### paddle ocr approach

# def getPaddle(img_path):
#   ocr_model = PaddleOCR(lang='en')
#   %matplotlib inline
#   result = ocr_model.ocr(img_path)
#   # draw result
#   result = result[0]
#   image = Image.open(img_path).convert('RGB')
#   boxes = [line[0] for line in result]
#   txts = [line[1][0] for line in result]
#   scores = [line[1][1] for line in result]
#   font_path = os.path.join(path, 'fonts/', 'latin.ttf')
#   print(txts)
#   # Visualize our image and detections
#   plt.figure(figsize=(25,18))
#   im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path)
#   im_show = Image.fromarray(im_show)
#   plt.imshow(im_show) 

# filename = imgnumber + '.jpg'
# img_path = jpg_path + filename
# getPaddle(img_path)

In [6]:
# #### paddleOCR Image extraction 
"""loading all images"""
def loadImagesFromjpg(directory: str = 'files/jpg'):
    print("loading invoices from", directory)
    subDirectories = os.listdir(directory)
    images = dict()
    for subdir in subDirectories:
        path = f'{directory}/{subdir}'
        if not os.path.isdir(path):
            continue
        files = os.listdir(path)
        for file in files:
            if not file.endswith('.jpg'):
                continue
            filename = f'{path}/{file}'
            images[filename] = subdir

    return getImgText(images)

"""extracting text from images"""
def getImgText(images):
  """dictionary of image text"""
  image_text = {}
  for image, type in images.items():
    ocr_model = PaddleOCR(lang='en', show_log=False)
    sentence = ''
    result = ocr_model.ocr(image)
    result = result[0]
    for res in result:
        sentence += ' ' + res[1][0] + ' '
    image_text[image[-95:]] = sentence
  return image_text

image_text = loadImagesFromjpg(path +'files/jpg')
image_text

In [ ]:
#### creating dataframe from extracted text dictionary
image_text_df = pd.DataFrame.from_dict(image_text,orient='index').reset_index()
image_text_df.rename(columns = {'index':'image_type',0:'extracted_text'},inplace = True)
image_text_df.head()

<IPython.core.display.Javascript object>

,image_type,extracted_text
0,drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Fi...,MAHAVEERS CHEFS CHOICE 10227 KING GEORGE BLV...
1,drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Fi...,BHARAT PARMAR SWEETS & 13745 72 AVE UNIT V3...
2,drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Fi...,NAHM THAI BISTRO 650AVE UNT V3T SURREY BC ...
3,rive/MyDrive/Colab Notebooks/CPSE 4830 OCR Fin...,PACIFIC CABS CAR 44 3-17943 55 AVE V3S6C4 ...
4,drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Fi...,TOMOKAZU 1128 W BROADWAY #2 V6H1G5 VANCOUVE...


extract csv to work locally

In [ ]:
#### train test split
random.seed(234)
# Shuffle your dataset 
shuffle_df = image_text_df.sample(frac=1)
# Define a size for your train set 
train_size = int(0.7 * len(image_text_df))
# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

In [ ]:
#### saving dataframe for local runtime
train_set.to_csv('/content/drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Final Project/installation files/files/derived dataframes/train_df.csv')
test_set.to_csv('/content/drive/MyDrive/Colab Notebooks/CPSE 4830 OCR Final Project/installation files/files/derived dataframes/test_df.csv')